In [2]:
si_id = [1,6,4,2,5,3,7,8,31,32,33,34,35,36,37,38,39]

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(1,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', 'area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  (주)한국다원호간보 부산지사 [한국관광 품질인증/Korea Quality]
부산 중구
부산광역시 중구 대청로155번길 6


no 2 :  핫트랙스 (부산점)
부산 부산진구
부산광역시 부산진구 중앙대로 658


no 3 :  정관장 (롯데백화점 센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 59


no 4 :  동보프라자
부산 부산진구
부산광역시 부산진구 중앙대로 704


no 5 :  오휘 (롯데백화점센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 59


no 6 :  분더샵 (신세계백화점 센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 35


no 7 :  서면 지하도상가
부산 부산진구
부산광역시 부산진구 중앙대로 737


no 8 :  애플아울렛
부산 사상구
부산광역시 사상구 사상로 201


no 9 :  광복 지하도상가
부산 중구
부산광역시 중구 중앙대로29번길 8


no 10 :  남포동 지하도상가
부산 중구
부산광역시 중구 구덕로 지하 44


no 11 :  핫트랙스 (대구점)
대구 중구
대구광역시 중구 국채보상로 586


no 12 :  동아아울렛 (본점)
대구 중구
대구광역시 중구 경상감영길 171


no 13 :  대구 서문시장 & 서문시장 야시장
대구 중구
대구광역시 중구 큰장로26길 45




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(1,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', 'area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  (주)한국다원호간보 부산지사 [한국관광 품질인증/Korea Quality]
부산 중구
부산광역시 중구 대청로155번길 6


no 2 :  핫트랙스 (부산점)
부산 부산진구
부산광역시 부산진구 중앙대로 658


no 3 :  정관장 (롯데백화점 센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 59


no 4 :  동보프라자
부산 부산진구
부산광역시 부산진구 중앙대로 704


no 5 :  오휘 (롯데백화점센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 59


no 6 :  분더샵 (신세계백화점 센텀시티점)
부산 해운대구
부산광역시 해운대구 센텀남대로 35


no 7 :  서면 지하도상가
부산 부산진구
부산광역시 부산진구 중앙대로 737


no 8 :  애플아울렛
부산 사상구
부산광역시 사상구 사상로 201


no 9 :  광복 지하도상가
부산 중구
부산광역시 중구 중앙대로29번길 8


no 10 :  남포동 지하도상가
부산 중구
부산광역시 중구 구덕로 지하 44


no 11 :  핫트랙스 (대구점)
대구 중구
대구광역시 중구 국채보상로 586


no 12 :  동아아울렛 (본점)
대구 중구
대구광역시 중구 경상감영길 171


no 13 :  대구 서문시장 & 서문시장 야시장
대구 중구
대구광역시 중구 큰장로26길 45




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(4,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', 'area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  올리브 아울렛 (인천점)
인천 남동구
인천광역시 남동구 남동대로 880


no 2 :  인천 모래내시장
인천 남동구
인천광역시 남동구 호구포로810번길 42-8


no 3 :  엔타스면세점
인천 중구
인천광역시 중구 영종해안남로321번길 186




IndexError: list index out of range

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(4,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  광주문화관광상품전시장 광주아트샵
광주 동구
광주광역시 동구 예술길 18


no 2 :  남광주시장(남광주밤기차야시장)
광주 동구
광주광역시 동구 양림로 117


no 3 :  광주 대인시장 (대인예술시장)
광주 동구
광주광역시 동구 제봉로194번길 7-1


no 4 :  인당국악사
광주 동구
광주광역시 동구 제봉로 75


no 5 :  광주 양동시장
광주 서구
광주광역시 서구 천변좌로 238


no 6 :  1913송정역시장
광주 광산구
광주광역시 광산구 송정로16번길 38


no 7 :  금남지하도상가
광주 동구
광주광역시 동구 서석로 지하 56


no 8 :  예술개미장터
광주 동구
광주광역시 동구 예술길 24


no 9 :  말바우장 / 말바우시장 (2, 4, 7, 9일)
광주 북구
광주광역시 북구 동문대로85번길 62


no 10 :  롯데아울렛 광주월드컵점
광주 서구
광주광역시 서구 금화로 240


no 11 :  은행동 으능정이 문화의거리
대전 중구
대전광역시 중구 중앙로 170


no 12 :  대전 중앙시장
대전 동구
대전광역시 동구 대전로 783


no 13 :  대전 태평시장
대전 중구
대전광역시 중구 수침로55번길 56


no 14 :  패션월드
대전 서구
대전광역시 서구 월평새뜸로 5


no 15 :  둔산전자타운
대전 서구
대전광역시 서구 도산로403번길 21


no 16 :  대전 중리전통시장
대전 대덕구
대전광역시 대덕구 중리북로37번길 26


no 17 :  갤러리아 백화점 (타임월드점)
대전 서구
대전광역시 서구 대덕대로 211


no 18 :  유성장 (4, 9일)
대전 유성구
대전광역시 유성구 유성대로730번길 24


no 19 :  신탄진장 (3, 8일)
대전 대덕구
대전광역시 대덕구 석봉로37번길 47


no 20 :  역전시장
대전 동구
대전광역시 동구 역전시장길 34


no 21 :  신정 평화시장
울산 남구
울산광역시 남구 봉월로67번길 17


no 22 :  울산 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="contents"]/div[2]/div[1]/ul/li[2]/div[2]/div/a"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰2'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(1,4):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  (주)한국다원호간보 부산지사 [한국관광 품질인증/Korea Quality]
부산 중구
부산광역시 중구 대청로155번길 6




KeyboardInterrupt: 

In [6]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰2'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(5,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  은행동 으능정이 문화의거리
대전 중구
대전광역시 중구 중앙로 170


no 2 :  대전 중앙시장
대전 동구
대전광역시 동구 대전로 783


no 3 :  대전 태평시장
대전 중구
대전광역시 중구 수침로55번길 56


no 4 :  패션월드
대전 서구
대전광역시 서구 월평새뜸로 5


no 5 :  둔산전자타운
대전 서구
대전광역시 서구 도산로403번길 21


no 6 :  대전 중리전통시장
대전 대덕구
대전광역시 대덕구 중리북로37번길 26


no 7 :  갤러리아 백화점 (타임월드점)
대전 서구
대전광역시 서구 대덕대로 211


no 8 :  유성장 (4, 9일)
대전 유성구
대전광역시 유성구 유성대로730번길 24


no 9 :  신탄진장 (3, 8일)
대전 대덕구
대전광역시 대덕구 석봉로37번길 47


no 10 :  역전시장
대전 동구
대전광역시 동구 역전시장길 34


no 11 :  신정 평화시장
울산 남구
울산광역시 남구 봉월로67번길 17


no 12 :  울산 농수산물도매시장
울산 남구
울산광역시 남구 삼산로 324


no 13 :  울산 중앙시장
울산 중구
울산광역시 중구 번영로 329


no 14 :  성남프라자
울산 중구
울산광역시 중구 중앙3길 6


no 15 :  울산 진산면세점
울산 중구
울산광역시 중구 번영로 363


no 16 :  호계장 / 호계공설시장 (1,6일)
울산 북구
울산광역시 북구 호계3길 17-11


no 17 :  언양장 / 언양알프스시장 (2, 7일)
울산 울주군
울산광역시 울주군 언양읍 장터1길 22


no 18 :  방어진 활어센터
울산 동구
울산광역시 동구 중진2길 5


no 19 :  남목시장
울산 동구
울산광역시 동구 남목17길 9


no 20 :  울산 산업공구월드
울산 남구
울산광역시 남구 갈밭로 24


no 21 :  세종전통시장 (4, 9일)
세종 한솔동
세종특별자치시 조치원읍 조치원8길 42




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="contents"]/div[2]/div[1]/ul/li[2]/div[2]/div/a"}
  (Session info: chrome=85.0.4183.102)
  (Driver info: chromedriver=2.40.565498 (ea082db3280dd6843ebfb08a625e3eb905c4f5ab),platform=Windows NT 10.0.18363 x86_64)


In [7]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰2'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(9,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  동쪽바다중앙시장
강원 동해시
강원도 동해시 중앙시장길 12


no 2 :  아남프라자
강원 속초시
강원도 속초시 중앙로5번길 8


no 3 :  삼척 중앙시장 (2, 7일)
강원 삼척시
강원도 삼척시 진주로 12-21


no 4 :  주천시장 (1, 6일)
강원 영월군
강원도 영월군 주천면 주천시장길 47-5


no 5 :  사직번개시장 (삼척 번개시장)
강원 삼척시
강원도 삼척시 중앙로 10


no 6 :  원주 단구시장
강원 원주시
강원도 원주시 송림길 48-13


no 7 :  강릉 서부시장
강원 강릉시
강원도 강릉시 임영로155번길 6


no 8 :  원주 남부시장
강원 원주시
강원도 원주시 치악로 1803


no 9 :  원주 농산물도매시장
강원 원주시
강원도 원주시 서원대로 33


no 10 :  춘천 풍물시장 / 풍물장 (2, 7일)
강원 춘천시
강원도 춘천시 영서로 2352-20


no 11 :  광혜원장 (3, 8일)
충북 진천군
충청북도 진천군 광혜원면 장기1안길 16


no 12 :  복드림 도깨비방망이
충북 제천시
충청북도 제천시 백운면 금봉로 142


no 13 :  용산장 (5, 10일)
충북 영동군
충청북도 영동군 용산면 용산로 338


no 14 :  영동장 (4, 9일)
충북 영동군
충청북도 영동군 영동읍 영동시장2길 10-1


no 15 :  옥산장 (3, 8일)
충북 청주시
충북 청주시 흥덕구 옥산면 오산리


no 16 :  도예공방 필
충북 충주시
충청북도 충주시 연원5길 26


no 17 :  진천장 (5, 10일)
충북 진천군
충청북도 진천군 진천읍 남산10길 27


no 18 :  삼화대장간
충북 충주시
충청북도 충주시 충인1길 12


no 19 :  단양장 / 단양구경시장 (1, 6일)
충북 단양군
충청북도 단양군 단양읍 도전5길 31


no 20 :  충주 민속5일장 / 풍물시장 (5, 10일)
충북 충주시
충청북도 충주시 충인1길 22


no 21 :  전통예산옹기

AttributeError: 'NoneType' object has no attribute 'find'

In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt

import random
import os  

from selenium.webdriver.support.ui import Select

query_txt = '쇼핑몰2'
# 파일 저장
f_dir = 'c:\\temp\\'
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'


s_time = time.time( )

path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%87%BC%ED%95%91&choiceTagId=0f29b431-75ac-4ab4-a892-b247d516b31d')

time.sleep(random.randrange(4,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

no2=[]           # 게시글 번호 컬럼
contents=[ ]     # 게시글 제목 컬럼
area_CD2=[]      # 게시글 행정동코드 컬럼
addr2 = []      # 게시글 주소 컬럼
no = 1

for j in range(14,17):
    # 하나 클릭 - 도 선택
    first = driver.find_element_by_id('arealist')
    first.find_element_by_xpath('''//*[@id=%s]/button'''%si_id[j]).click()
    cnt = 1
    time.sleep(3)
#     for i in si_list:  
#         re = i
    #         f = open(ff_name, 'a',encoding='UTF-8')
#     while(cnt != 17):
        # 구 클릭 - 구 선택
#         time.sleep(2)
#         t = driver.find_element_by_id('sigungulist')
#         time.sleep(2)
#         t.find_element_by_xpath('''//*[@id="%d"]/button'''%re[cnt]).click()
#         time.sleep(2)
    for k in range(1,11):
        f = open(ff_name, 'a', encoding='UTF-8')
       
        #각 게시글의 제목 누르기    
        title = driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).text
        driver.find_element_by_xpath("""//*[@id="contents"]/div[2]/div[1]/ul/li[%d]/div[2]/div/a""" % k).click()
        time.sleep(random.randrange(5,7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')        
        content_list = soup.find('div','wrap_contView')    

#         title = driver.find_element_by_xpath('''//*[@id="topTitle"]''').text
#         title = soup.find('h2').get_text()
        area_CD = soup.find('div', class_='area_address').find('span').get_text()
        try:
            addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[3]''').text
            addr = addr.split('주소 ')
            addr = addr[1]
            
        except:
            if (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif (driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            elif(driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text.split('주소 ')):
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[1]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
            else:
                addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[4]''').text
                addr = addr.split('주소 ')
                addr = addr[1]
#             addr = driver.find_element_by_xpath('''//*[@id="detailGo"]/div[3]/div/div[1]/div/ul/li[2]''').text
#             addr = addr.split('주소 ')
#             addr = addr[1]
        print("no", no, ": ", title)
        print(area_CD)
        print(addr)
        f.write(str(no) + '/'+ str(title) + '/'+str(area_CD) +'/'+ str(addr) + "\n")
        f.close( )

        print("\n")
        no += 1
        no2.append(no)
        contents.append(title)
        area_CD2.append(area_CD)
        addr2.append(addr)

        driver.back( )  # 뒤로 돌아가기 기능
        time.sleep(random.randrange(5,7))
    cnt += 1
    
tour = pd.DataFrame()
tour['no']=no2
tour['title']=contents
tour['area_cd']=area_CD2
tour['addr']=addr2

# csv 형태로 저장하기
tour.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
tour.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

# Step 9. 요약정보 보여주기
print("\n") 
print("=" *80)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %ff_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )
        

no 1 :  남원장 / 남원공설시장 (4, 9일)
전북 남원시
전라북도 남원시 의총로 51


no 2 :  온고을공예방
전북 전주시
전라북도 전주시 완산구 향교길 51-9


no 3 :  익산 주얼팰리스
전북 익산시
전라북도 익산시 왕궁면 호반로 8


no 4 :  남원 목운공예사
전북 남원시
전라북도 남원시 요천로 1230-9


no 5 :  신영시장
전북 군산시
전라북도 군산시 동신영길 36


no 6 :  고창장 (3, 8일)
전북 고창군
전라북도 고창군 고창읍 동산5길 24


no 7 :  곰소젓갈단지 (구시장)
전북 부안군
전라북도 부안군 진서면 곰소항길 22-11


no 8 :  오수상설시장 / 오수5일장 (5, 10일)
전북 임실군
전라북도 임실군 오수면 오수로 159


no 9 :  무주반딧불시장(무주반딧불장터) (1, 6일)
전북 무주군
전라북도 무주군 무주읍 장터로 2


no 10 :  전주 남부시장
전북 전주시
전라북도 전주시 완산구 풍남문2길 63


no 11 :  충주산방
전남 나주시
전라남도 나주시 공산면 금강로 253


no 12 :  나주목물 (나주반장 김춘식)
전남 나주시
전라남도 나주시 영산로 5116-12


no 13 :  목포 신중앙시장
전남 목포시
전라남도 목포시 원산중앙로 51


no 14 :  흥진상사
전남 여수시
전라남도 여수시 이순신광장로 49


no 15 :  광양 청매실농원
전남 광양시
전라남도 광양시 다압면 지막1길 55


no 16 :  동박새꿈정원
전남 여수시
전라남도 여수시 오동도로 238-32


no 17 :  보향다원
전남 보성군
전라남도 보성군 보성읍 동암1길 144


no 18 :  여수 특산품전시판매장
전남 여수시
전라남도 여수시 봉산남9길 30-12


no 19 :  여수 돌산갓장터마을갓김치
전남 여수시
전라남도 여수시 돌산읍 향일암로 359


no 20 :  구례5일장 / 구례전통시장(3, 8일)
전남 구례군
전라남도 구례군 구례읍 5일시장작은길 20


n